In [44]:
import urllib
import urllib.request
from urllib.parse import urlparse
from bs4 import BeautifulSoup
import requests
import re #regex
from selenium import webdriver
from csv import DictWriter
import json

import numpy as np
import pandas as pd

import shutil
import tempfile

In [60]:
#First level urls, for month 1-12, year 1998-2021, possible page range: 1-4
url_list=[]
for i in range(1,13):
    for j in range (1998, 2022):
        for k in range(1,4):
            urll='http://www.christies.com/results/?month='+str(i)+'&year='+str(j)+'&locations=&scids=&action=paging&initialpageload=false&pg='+str(k)
            url_list.append(urll)
            #print urll
print(len(url_list))


864


In [61]:
url_list

['http://www.christies.com/results/?month=1&year=1998&locations=&scids=&action=paging&initialpageload=false&pg=1',
 'http://www.christies.com/results/?month=1&year=1998&locations=&scids=&action=paging&initialpageload=false&pg=2',
 'http://www.christies.com/results/?month=1&year=1998&locations=&scids=&action=paging&initialpageload=false&pg=3',
 'http://www.christies.com/results/?month=1&year=1999&locations=&scids=&action=paging&initialpageload=false&pg=1',
 'http://www.christies.com/results/?month=1&year=1999&locations=&scids=&action=paging&initialpageload=false&pg=2',
 'http://www.christies.com/results/?month=1&year=1999&locations=&scids=&action=paging&initialpageload=false&pg=3',
 'http://www.christies.com/results/?month=1&year=2000&locations=&scids=&action=paging&initialpageload=false&pg=1',
 'http://www.christies.com/results/?month=1&year=2000&locations=&scids=&action=paging&initialpageload=false&pg=2',
 'http://www.christies.com/results/?month=1&year=2000&locations=&scids=&action=p

In [62]:
url_list_df = pd.DataFrame({'url':url_list})
# url_list_df.to_csv(r'url_list.csv', index = False, header = True)

In [63]:
url_list_df

,url
0,http://www.christies.com/results/?month=1&year...
1,http://www.christies.com/results/?month=1&year...
2,http://www.christies.com/results/?month=1&year...
3,http://www.christies.com/results/?month=1&year...
4,http://www.christies.com/results/?month=1&year...
...,...
859,http://www.christies.com/results/?month=12&yea...
860,http://www.christies.com/results/?month=12&yea...
861,http://www.christies.com/results/?month=12&yea...
862,http://www.christies.com/results/?month=12&yea...


In [49]:
#Step 2. Reading url into html files
tmp_list=[]

for url in url_list: 
    #The try block lets you test a block of code for errors.
    try:
        auction = urllib.urlopen(url) #open the link page     
        html = auction.read() # returns the specified number of bytes from the file
        soup = BeautifulSoup(html)
        grand_list = soup.find("ul", {"id": "list-items"})
        
        #creating a sub_soup
        sub_soup = BeautifulSoup(str(grand_list))
        events = sub_soup.find_all("li", id = re.compile('^day-')) 
        print('.')
    
    #The except block lets you handle the error.
    except:
        print (url)
        pass
    print('An exception occurred: ','-')
    
    #inner loop to construct a dictionary for each event:
    final_result = []
    for event in events:
        try:
            auction = {}
            url_find = event.find("a", {"class": "description"}).get('href').encode('ascii', 'ignore').strip()
            auction['month'] = event.find("span", "month").get_text().encode('ascii', 'ignore').strip()
            auction['date'] = event.find("span", "date").get_text().encode('ascii', 'ignore').strip()
            auction['year'] =  event.find("span", "year").get_text().encode('ascii', 'ignore').strip()
            auction['ID'] = event.find("a", {"class": "sale-number"}).get_text().encode('ascii', 'ignore').strip()
            auction['location'] = event.find("span", "location").get_text().encode('ascii', 'ignore').strip()
            auction['event_name'] = event.find("a", {"class": "description"}).get_text().encode('ascii', 'ignore').strip()
            auction['total_sales'] = event.find("ul", {"class": "auction-links"}).find('a').get_text().encode('ascii', 'ignore').strip()
            auction['url'] = "http://www.christies.com" + url_find
            final_result.append(auction)
            #
            print (auction['month'])
        except Exception as e:
            print (e)
    tmp_list += final_result
    
print (len(tmp_list))

http://www.christies.com/results/?month=1&year=1998&locations=&scids=&action=paging&initialpageload=false&pg=1
An exception occurred:  -
http://www.christies.com/results/?month=1&year=1998&locations=&scids=&action=paging&initialpageload=false&pg=2
An exception occurred:  -
http://www.christies.com/results/?month=1&year=1998&locations=&scids=&action=paging&initialpageload=false&pg=3
An exception occurred:  -
http://www.christies.com/results/?month=1&year=1999&locations=&scids=&action=paging&initialpageload=false&pg=1
An exception occurred:  -
http://www.christies.com/results/?month=1&year=1999&locations=&scids=&action=paging&initialpageload=false&pg=2
An exception occurred:  -
http://www.christies.com/results/?month=1&year=1999&locations=&scids=&action=paging&initialpageload=false&pg=3
An exception occurred:  -
http://www.christies.com/results/?month=1&year=2000&locations=&scids=&action=paging&initialpageload=false&pg=1
An exception occurred:  -
http://www.christies.com/results/?month=1

An exception occurred:  -
http://www.christies.com/results/?month=10&year=2009&locations=&scids=&action=paging&initialpageload=false&pg=3
An exception occurred:  -
http://www.christies.com/results/?month=10&year=2010&locations=&scids=&action=paging&initialpageload=false&pg=1
An exception occurred:  -
http://www.christies.com/results/?month=10&year=2010&locations=&scids=&action=paging&initialpageload=false&pg=2
An exception occurred:  -
http://www.christies.com/results/?month=10&year=2010&locations=&scids=&action=paging&initialpageload=false&pg=3
An exception occurred:  -
http://www.christies.com/results/?month=10&year=2011&locations=&scids=&action=paging&initialpageload=false&pg=1
An exception occurred:  -
http://www.christies.com/results/?month=10&year=2011&locations=&scids=&action=paging&initialpageload=false&pg=2
An exception occurred:  -
http://www.christies.com/results/?month=10&year=2011&locations=&scids=&action=paging&initialpageload=false&pg=3
An exception occurred:  -
http://w